## 00_DATA_PREPROCESSING

## 1 - Import packages and Data

In [45]:
import numpy as np
import scipy as sp
import sklearn as sk
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb

Code below for using GPU, can be ignored if numba isn't installed for computer doesn't have a strong graphics card.

In [46]:
from numba import jit, cuda

In [47]:
pd.set_option('display.max_columns', 200)

In [48]:
columns_df = pd.read_csv('../raw_data/_columns.csv')
columns_df

,ColumnName,ColumnDescription
0,Address,website address the data is scraped from \n
1,League,League or Tournament the match took place in \n
2,Year,Year the match took place in \n
3,Season,Spring or Summer depending on which half of th...
4,Type,"Season, Playoffs, Regional, or International m..."
5,blueTeamTag,Blue Team's tag name (ex. Team SoloMid is TSM...
6,bResult,Result of the match for Blue Team - 1 is a wi...
7,rResult,Result of the match for Red Team - 1 is a win...
8,redTeamTag,Red Team's Tag Name (ex. Team SoloMid is TSM) \n
9,gamelength,Game length in minutes \n


In [49]:
bans_df = pd.read_csv('../raw_data/bans.csv')
print(bans_df.shape)
bans_df.head()

(15240, 7)


,Address,Team,ban_1,ban_2,ban_3,ban_4,ban_5
0,http://matchhistory.na.leagueoflegends.com/en/...,blueBans,Rumble,Kassadin,Lissandra,NaN,NaN
1,http://matchhistory.na.leagueoflegends.com/en/...,blueBans,Kassadin,Sivir,Lissandra,NaN,NaN
2,http://matchhistory.na.leagueoflegends.com/en/...,blueBans,JarvanIV,Lissandra,Kassadin,NaN,NaN
3,http://matchhistory.na.leagueoflegends.com/en/...,blueBans,Annie,Lissandra,Kassadin,NaN,NaN
4,http://matchhistory.na.leagueoflegends.com/en/...,blueBans,Irelia,Pantheon,Kassadin,NaN,NaN


In [50]:
gold_df = pd.read_csv('../raw_data/gold.csv')
print(gold_df.shape)
gold_df.head()

(99060, 97)


,Address,Type,min_1,min_2,min_3,min_4,min_5,min_6,min_7,min_8,min_9,min_10,min_11,min_12,min_13,min_14,min_15,min_16,min_17,min_18,min_19,min_20,min_21,min_22,min_23,min_24,min_25,min_26,min_27,min_28,min_29,min_30,min_31,min_32,min_33,min_34,min_35,min_36,min_37,min_38,min_39,min_40,min_41,min_42,min_43,min_44,min_45,min_46,min_47,min_48,min_49,min_50,min_51,min_52,min_53,min_54,min_55,min_56,min_57,min_58,min_59,min_60,min_61,min_62,min_63,min_64,min_65,min_66,min_67,min_68,min_69,min_70,min_71,min_72,min_73,min_74,min_75,min_76,min_77,min_78,min_79,min_80,min_81,min_82,min_83,min_84,min_85,min_86,min_87,min_88,min_89,min_90,min_91,min_92,min_93,min_94,min_95
0,http://matchhistory.na.leagueoflegends.com/en/...,golddiff,0,0,-14,-65,-268,-431,-488,-789,-494,-625,-1044,-313,-760,-697,-790,-611,240,845.0,797.0,1422.0,987.0,169.0,432.0,491.0,1205.0,1527.0,1647.0,1847.0,3750.0,4719.0,3561.0,3367.0,2886.0,2906.0,4411.0,4473.0,4639.0,4762.0,4686.0,6057.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,http://matchhistory.na.leagueoflegends.com/en/...,golddiff,0,0,-26,-18,147,237,-152,18,88,-242,102,117,802,1420,1394,1301,1489,1563.0,1368.0,1105.0,205.0,192.0,587.0,377.0,667.0,415.0,1876.0,1244.0,2130.0,2431.0,680.0,1520.0,949.0,1894.0,2644.0,3394.0,3726.0,1165.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,http://matchhistory.na.leagueoflegends.com/en/...,golddiff,0,0,10,-60,34,37,589,1064,1258,913,1233,1597,1575,3046,2922,3074,3626,3466.0,5634.0,5293.0,4597.0,4360.0,4616.0,4489.0,4880.0,5865.0,6993.0,7049.0,7029.0,7047.0,7160.0,7081.0,7582.0,9917.0,10337.0,9823.0,12307.0,13201.0,13924.0,13915.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,http://matchhistory.na.leagueoflegends.com/en/...,golddiff,0,0,-15,25,228,-6,-243,175,-346,16,-258,-57,-190,-111,-335,-8,324,428.0,-124.0,768.0,2712.0,1813.0,198.0,1242.0,1245.0,1278.0,1240.0,-664.0,-1195.0,-1157.0,-2161.0,-2504.0,-3873.0,-3688.0,-3801.0,-3668.0,-3612.0,-5071.0,-2825.0,-1464.0,-4507.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,http://matchhistory.na.leagueoflegends.com/en/...,golddiff,40,40,44,-36,113,158,-121,-191,23,205,156,272,-271,-896,-574,177,-425,-730.0,-318.0,478.0,926.0,761.0,-286.0,473.0,490.0,1265.0,2526.0,3890.0,4319.0,5121.0,5140.0,5141.0,6866.0,9517.0,11322.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [51]:
kills_df = pd.read_csv('../raw_data/kills.csv')
print(kills_df.shape)
kills_df.head()

(191069, 11)


,Address,Team,Time,Victim,Killer,Assist_1,Assist_2,Assist_3,Assist_4,x_pos,y_pos
0,http://matchhistory.na.leagueoflegends.com/en/...,bKills,10.820,C9 Hai,TSM Bjergsen,NaN,NaN,NaN,NaN,9229,8469
1,http://matchhistory.na.leagueoflegends.com/en/...,bKills,16.286,C9 LemonNation,TSM WildTurtle,TSM Santorin,TSM Bjergsen,TSM Lustboy,NaN,13908,5960
2,http://matchhistory.na.leagueoflegends.com/en/...,bKills,18.733,C9 Hai,TSM Bjergsen,TSM Santorin,TSM WildTurtle,TSM Lustboy,NaN,8281,7889
3,http://matchhistory.na.leagueoflegends.com/en/...,bKills,18.880,C9 Meteos,TSM Dyrus,TSM Santorin,TSM Bjergsen,TSM WildTurtle,TSM Lustboy,8522,8895
4,http://matchhistory.na.leagueoflegends.com/en/...,bKills,27.005,C9 Balls,TSM Bjergsen,TSM Dyrus,TSM Santorin,TSM WildTurtle,TSM Lustboy,8902,7643


In [52]:
monsters_df = pd.read_csv('../raw_data/monsters.csv')
print(monsters_df.shape)
monsters_df.head()

(44248, 4)


,Address,Team,Time,Type
0,http://matchhistory.na.leagueoflegends.com/en/...,bDragons,37.267,DRAGON
1,http://matchhistory.na.leagueoflegends.com/en/...,bDragons,32.545,DRAGON
2,http://matchhistory.na.leagueoflegends.com/en/...,bDragons,26.177,DRAGON
3,http://matchhistory.na.leagueoflegends.com/en/...,bDragons,19.119,DRAGON
4,http://matchhistory.na.leagueoflegends.com/en/...,bDragons,24.577,DRAGON


In [53]:
structures_df = pd.read_csv('../raw_data/structures.csv')
print(structures_df.shape)
structures_df.head()

(121386, 5)


,Address,Team,Time,Lane,Type
0,http://matchhistory.na.leagueoflegends.com/en/...,bTowers,27.542,MID_LANE,BASE_TURRET
1,http://matchhistory.na.leagueoflegends.com/en/...,bTowers,39.269,MID_LANE,NEXUS_TURRET
2,http://matchhistory.na.leagueoflegends.com/en/...,bTowers,33.583,BOT_LANE,INNER_TURRET
3,http://matchhistory.na.leagueoflegends.com/en/...,bTowers,23.038,TOP_LANE,INNER_TURRET
4,http://matchhistory.na.leagueoflegends.com/en/...,bTowers,16.556,BOT_LANE,OUTER_TURRET


In [54]:
matchInfo_df = pd.read_csv('../raw_data/matchinfo.csv')
print(matchInfo_df.shape)
matchInfo_df.head()

(7620, 30)


,League,Year,Season,Type,blueTeamTag,bResult,rResult,redTeamTag,gamelength,blueTop,blueTopChamp,blueJungle,blueJungleChamp,blueMiddle,blueMiddleChamp,blueADC,blueADCChamp,blueSupport,blueSupportChamp,redTop,redTopChamp,redJungle,redJungleChamp,redMiddle,redMiddleChamp,redADC,redADCChamp,redSupport,redSupportChamp,Address
0,NALCS,2015,Spring,Season,TSM,1,0,C9,40,Dyrus,Irelia,Santorin,RekSai,Bjergsen,Ahri,WildTurtle,Jinx,Lustboy,Janna,Balls,Gnar,Meteos,Elise,Hai,Fizz,Sneaky,Sivir,LemonNation,Thresh,http://matchhistory.na.leagueoflegends.com/en/...
1,NALCS,2015,Spring,Season,CST,0,1,DIG,38,Cris,Gnar,Impaler,Rengar,Jesiz,Ahri,Mash,Caitlyn,Sheep,Leona,Gamsu,Irelia,Crumbzz,JarvanIV,Shiphtur,Azir,CoreJJ,Corki,KiWiKiD,Annie,http://matchhistory.na.leagueoflegends.com/en/...
2,NALCS,2015,Spring,Season,WFX,1,0,GV,40,Flaresz,Renekton,ShorterACE,Rengar,Pobelter,Fizz,Altec,Sivir,Gleeb,Annie,Hauntzer,Sion,Saintvicious,LeeSin,Keane,Azir,Cop,Corki,BunnyFuFuu,Janna,http://matchhistory.na.leagueoflegends.com/en/...
3,NALCS,2015,Spring,Season,TIP,0,1,TL,41,Rhux,Irelia,Rush,JarvanIV,XiaoWeiXiao,Leblanc,Apollo,Sivir,Adrian,Thresh,Quas,Gnar,IWDominate,Nunu,Fenix,Lulu,KEITH,KogMaw,Xpecial,Janna,http://matchhistory.na.leagueoflegends.com/en/...
4,NALCS,2015,Spring,Season,CLG,1,0,T8,35,Benny,Gnar,Xmithie,JarvanIV,Link,Lissandra,Doublelift,Tristana,aphromoo,Janna,CaliTrlolz8,Sion,Porpoise8,RekSai,Slooshi8,Lulu,Maplestreet8,Corki,Dodo8,Annie,http://matchhistory.na.leagueoflegends.com/en/...


In [55]:
lol_df = pd.read_csv('../raw_data/LeagueofLegends.csv')
print(lol_df.shape)
lol_df.head()

(7620, 57)


,League,Year,Season,Type,blueTeamTag,bResult,rResult,redTeamTag,gamelength,golddiff,goldblue,bKills,bTowers,bInhibs,bDragons,bBarons,bHeralds,goldred,rKills,rTowers,rInhibs,rDragons,rBarons,rHeralds,blueTop,blueTopChamp,goldblueTop,blueJungle,blueJungleChamp,goldblueJungle,blueMiddle,blueMiddleChamp,goldblueMiddle,blueADC,blueADCChamp,goldblueADC,blueSupport,blueSupportChamp,goldblueSupport,blueBans,redTop,redTopChamp,goldredTop,redJungle,redJungleChamp,goldredJungle,redMiddle,redMiddleChamp,goldredMiddle,redADC,redADCChamp,goldredADC,redSupport,redSupportChamp,goldredSupport,redBans,Address
0,NALCS,2015,Spring,Season,TSM,1,0,C9,40,"[0, 0, -14, -65, -268, -431, -488, -789, -494,...","[2415, 2415, 2711, 3887, 5068, 6171, 7412, 866...","[[10.82, 'C9 Hai', 'TSM Bjergsen', [], 9229, 8...","[[27.542, 'MID_LANE', 'BASE_TURRET'], [39.269,...","[[36.686, 'MID_LANE'], [29.274, 'MID_LANE']]","[[37.267, None]]",[],[],"[2415, 2415, 2725, 3952, 5336, 6602, 7900, 945...","[[16.529, 'TSM Lustboy', 'C9 Balls', ['C9 Mete...","[[39.23, 'TOP_LANE', 'INNER_TURRET'], [20.681,...",[],"[[17.14, None], [30.934, None], [24.641, None]]",[[29.954]],[],Dyrus,Irelia,"[475, 475, 532, 687, 893, 1058, 1172, 1471, 18...",Santorin,RekSai,"[475, 475, 532, 870, 1049, 1276, 1596, 1815, 2...",Bjergsen,Ahri,"[475, 475, 532, 807, 1102, 1307, 1651, 1950, 2...",WildTurtle,Jinx,"[475, 475, 532, 797, 1127, 1453, 1766, 2044, 2...",Lustboy,Janna,"[515, 515, 583, 726, 897, 1077, 1227, 1381, 15...","['Rumble', 'Kassadin', 'Lissandra']",Balls,Gnar,"[475, 475, 532, 728, 958, 1284, 1526, 1912, 21...",Meteos,Elise,"[475, 475, 532, 898, 1192, 1429, 1819, 2107, 2...",Hai,Fizz,"[475, 475, 552, 842, 1178, 1378, 1635, 1949, 2...",Sneaky,Sivir,"[475, 475, 532, 762, 1097, 1469, 1726, 2112, 2...",LemonNation,Thresh,"[515, 515, 577, 722, 911, 1042, 1194, 1370, 14...","['Tristana', 'Leblanc', 'Nidalee']",http://matchhistory.na.leagueoflegends.com/en/...
1,NALCS,2015,Spring,Season,CST,0,1,DIG,38,"[0, 0, -26, -18, 147, 237, -152, 18, 88, -242,...","[2415, 2415, 2705, 4108, 5511, 6797, 7637, 895...","[[11.104, 'DIG Shiphtur', 'CST Jesiz', ['CST I...","[[23.239, 'BOT_LANE', 'OUTER_TURRET'], [33.018...",[],"[[32.545, None], [26.177, None], [19.119, None]]",[[29.255]],[],"[2415, 2415, 2731, 4126, 5364, 6560, 7789, 893...","[[12.387, 'CST Jesiz', 'DIG Gamsu', ['DIG Ship...","[[19.257, 'MID_LANE', 'OUTER_TURRET'], [15.206...","[[36.813, 'MID_LANE']]","[[12.264, None]]",[],[],Cris,Gnar,"[475, 475, 532, 791, 1127, 1509, 1674, 1875, 2...",Impaler,Rengar,"[475, 475, 532, 895, 1176, 1334, 1447, 1859, 2...",Jesiz,Ahri,"[475, 475, 532, 816, 1102, 1413, 1624, 1937, 2...",Mash,Caitlyn,"[475, 475, 532, 856, 1182, 1468, 1653, 1890, 2...",Sheep,Leona,"[515, 515, 577, 750, 924, 1073, 1239, 1392, 15...","['Kassadin', 'Sivir', 'Lissandra']",Gamsu,Irelia,"[475, 475, 532, 811, 1042, 1237, 1625, 1852, 2...",Crumbzz,JarvanIV,"[475, 475, 532, 909, 1105, 1423, 1536, 1732, 1...",Shiphtur,Azir,"[475, 475, 552, 786, 1097, 1389, 1660, 1955, 2...",CoreJJ,Corki,"[475, 475, 532, 868, 1220, 1445, 1732, 1979, 2...",KiWiKiD,Annie,"[515, 515, 583, 752, 900, 1066, 1236, 1417, 15...","['RekSai', 'Janna', 'Leblanc']",http://matchhistory.na.leagueoflegends.com/en/...
2,NALCS,2015,Spring,Season,WFX,1,0,GV,40,"[0, 0, 10, -60, 34, 37, 589, 1064, 1258, 913, ...","[2415, 2415, 2726, 3794, 4933, 6236, 8109, 965...","[[5.255, 'GV Keane', 'WFX Pobelter', ['WFX Sho...","[[15.045, 'BOT_LANE', 'OUTER_TURRET'], [39.566...","[[37.511, 'TOP_LANE'], [37.38, 'BOT_LANE'], [3...","[[24.577, None], [37.867, None], [30.87, None]...",[[35.144]],[],"[2415, 2415, 2716, 3854, 4899, 6199, 7520, 859...","[[8.449, 'WFX Altec', 'GV Cop', ['GV BunnyFuFu...","[[24.62, 'MID_LANE', 'OUTER_TURRET'], [30.493,...",[],[],[],[],Flaresz,Renekton,"[475, 475, 533, 673, 828, 1075, 1428, 1775, 21...",ShorterACE,Rengar,"[475, 475, 543, 836, 1041, 1261, 1568, 2002, 2...",Pobelter,Fizz,"[475, 475, 533, 756, 1065, 1368, 2056, 2237, 2...",Altec,Sivir,"[475, 475, 533, 811, 1130

# 2 - Data cleaning and preprocessing

## Investigating NA's in each dataset (kills, gold, etc.)

NA's in Structures data. NA only occurs if none of a team's towers or inhibitors were destroyed that game, which is possible as not all are needed to be destroyed to win the game

In [56]:
structures_df.isna().sum()

Address       0
Team          0
Time       7606
Lane       7606
Type       7606
dtype: int64

Replacing the NA's. 0 for the numeric columns and none for the categorical.

In [60]:
structures_df = structures_df.fillna(value = {
    'Time': 0,
    'Lane': 'None',
    'Type': 'None'
})

NA's in the Bans data.
Any NA's would mean that the no one was chosen to be banned.

In [62]:
bans_df.isna().sum()

Address       0
Team          0
ban_1        48
ban_2        59
ban_3       139
ban_4      8276
ban_5      8362
dtype: int64

In [63]:
bans_df = bans_df.fillna(value = 'None')

NA's in the Monsters data.

In [65]:
monsters_df.isna().sum()

Address    0
Team       0
Time       0
Type       0
dtype: int64

NA's in the Kills data.

In [66]:
kills_df.isna().sum()

Address          0
Team             0
Time           113
Victim         113
Killer         113
Assist_1     12518
Assist_2     47887
Assist_3     96622
Assist_4    151355
x_pos          113
y_pos          113
dtype: int64

In [98]:
#kills_df[kills_df['Time'] == kills_df['Time'].isna()]
#kills_df.drop(kills_df.notna())
x = kills_df['Time'].isna()
#kills_df[x == 'True']

0         False
1         False
2         False
3         False
4         False
          ...  
191064    False
191065    False
191066    False
191067    False
191068    False
Name: Time, Length: 191069, dtype: bool

In [63]:
kills_df = kills_df.fillna(value = 'None')

Investigate NA's in gold dataframe. If game length was less than N minutes, gold for min_N and onwards are NAN, which makes sense.

In [81]:
# For example we expect the number of games less than 20 minutes to equal the number of and NA values for min_20.
print((gold_aug['gamelength'] < 20).sum())
gold_aug['min_20'].isna().sum()  # This is the case.

52


52

## New dataset structure

A look into the lol_df shows that it is a poorly put together dataset of all the other datasets. The code below is our attempt to recreate final data set to use. The main issue is that there are time based variables that aren't stored in a dataframe properly so it would be easier to just recreate a big data set to just instead.

First up is replacing two of the columns in the match info dataset that refer to who won. We are replacing those two with one column and instead of 0 or 1 to indicate the loser and winner, we will just replace it with Red or Blue. These refer to the teams during a match.

A key note is that all matches are identified by an address, so going forward we will be using that to match the game stats together.

In [83]:
address = pd.DataFrame(matchInfo_df['Address'])
address = pd.concat([address,matchInfo_df.iloc[:,0:4]],axis=1)
address = pd.concat([address,matchInfo_df.iloc[:,8]],axis=1)
address['Winner'] = matchInfo_df['bResult'].replace([1,0],['Blue','Red'])
address.columns = pd.MultiIndex.from_product([address.columns,['Info']])

address.head()

,Address,League,Year,Season,GameType,gamelength,Winner
,Info,Info,Info,Info,Info,Info,Info
0,http://matchhistory.na.leagueoflegends.com/en/...,NALCS,2015,Spring,Season,40,Blue
1,http://matchhistory.na.leagueoflegends.com/en/...,NALCS,2015,Spring,Season,38,Red
2,http://matchhistory.na.leagueoflegends.com/en/...,NALCS,2015,Spring,Season,40,Blue
3,http://matchhistory.na.leagueoflegends.com/en/...,NALCS,2015,Spring,Season,41,Red
4,http://matchhistory.na.leagueoflegends.com/en/...,NALCS,2015,Spring,Season,35,Blue


Seperating Blue team from Red Team

In [84]:
cols = matchInfo_df.columns.unique()
side1 = list()
side2 = list()
names = list()
for x in cols:
    if 'blue' in x:
        side1.append(x)
        names.append(x.replace('blue','',1))
    elif 'red' in x:
        side2.append(x)

In [85]:
team = matchInfo_df[side1]
team.columns = names
team.columns = pd.MultiIndex.from_product([team.columns, ['Blue']])
final_df = pd.concat([address,team], axis=1)

team = matchInfo_df[side2]
team.columns = names
team.columns = pd.MultiIndex.from_product([team.columns, ['Red']])
final_df = pd.concat([final_df,team], axis=1)

final_df = final_df.swaplevel(0,1,axis=1)
final_df.head()

Info                       \
                                             Address League  Year  Season   
0  http://matchhistory.na.leagueoflegends.com/en/...  NALCS  2015  Spring   
1  http://matchhistory.na.leagueoflegends.com/en/...  NALCS  2015  Spring   
2  http://matchhistory.na.leagueoflegends.com/en/...  NALCS  2015  Spring   
3  http://matchhistory.na.leagueoflegends.com/en/...  NALCS  2015  Spring   
4  http://matchhistory.na.leagueoflegends.com/en/...  NALCS  2015  Spring   

                                Blue                                 \
  GameType gamelength Winner TeamTag      Top  TopChamp      Jungle   
0   Season         40   Blue     TSM    Dyrus    Irelia    Santorin   
1   Season         38    Red     CST     Cris      Gnar     Impaler   
2   Season         40   Blue     WFX  Flaresz  Renekton  ShorterACE   
3   Season         41    Red     TIP     Rhux    Irelia        Rush   
4   Season         35   Blue     CLG    Benny      Gnar     Xmithie   

                                                                        \
  JungleChamp       Middle MiddleChamp         ADC  ADCChamp   Support   
0      RekSai     Bjergsen        Ahri  WildTurtle      Jinx   Lustboy   
1      Rengar        Jesiz        Ahri        Mash   Caitlyn     Sheep   
2      Rengar     Pobelter        Fizz       Altec     Sivir     Gleeb   
3    JarvanIV  XiaoWeiXiao     Leblanc      Apollo     Sivir    Adrian   
4    JarvanIV         Link   Lissandra  Doublelift  Tristana  aphromoo   

                   Red                                                  \
  SupportChamp TeamTag          Top TopChamp        Jungle JungleChamp   
0        Janna      C9        Balls     Gnar        Meteos       Elise   
1        Leona     DIG        Gamsu   Irelia       Crumbzz    JarvanIV   
2        Annie      GV     Hauntzer     Sion  Saintvicious      LeeSin   
3       Thresh      TL         Quas     Gnar    IWDominate        Nunu   
4        Janna      T8  CaliTrlolz8     Sion     Porpoise8      RekSai   

                                                                          
     Middle MiddleChamp           ADC ADCChamp      Support SupportChamp  
0       Hai        Fizz        Sneaky    Sivir  LemonNation       Thresh  
1  Shiphtur        Azir        CoreJJ    Corki      KiWiKiD        Annie  
2     Keane        Azir           Cop    Corki   BunnyFuFuu        Janna  
3     Fenix        Lulu         KEITH   KogMaw      Xpecial        Janna  
4  Slooshi8        Lulu  Maplestreet8    Corki        Dodo8        Annie

Adding Bans_df to the final_df. Bans is the only other data that isn't time based.

In [86]:
work = bans_df[bans_df['Team']=='blueBans']

address = pd.DataFrame(work['Address'])
address.columns = pd.MultiIndex.from_product([address.columns,['Info']])
work.columns = pd.MultiIndex.from_product([work.columns, ['Blue']])

work = pd.concat([address,work], axis=1)
work = work.drop(work.columns[1],axis=1)
work = work.drop(work.columns[1:2],axis=1)
work = work.swaplevel(0,1,axis=1)
work.head()
final_df = pd.merge(final_df,work)

In [87]:
work = bans_df[bans_df['Team'] != 'blueBans']

address = pd.DataFrame(work['Address'])
address.columns = pd.MultiIndex.from_product([address.columns, ['Info']])
work.columns = pd.MultiIndex.from_product([work.columns, ['Red']])

work = pd.concat([address,work], axis=1)
work = work.drop(work.columns[1], axis=1)
work = work.drop(work.columns[1:2], axis=1)
work = work.swaplevel(0, 1, axis=1)
#work.head()
final_df = pd.merge(final_df,work)

In [88]:
final_df.head()

Info                       \
                                             Address League  Year  Season   
0  http://matchhistory.na.leagueoflegends.com/en/...  NALCS  2015  Spring   
1  http://matchhistory.na.leagueoflegends.com/en/...  NALCS  2015  Spring   
2  http://matchhistory.na.leagueoflegends.com/en/...  NALCS  2015  Spring   
3  http://matchhistory.na.leagueoflegends.com/en/...  NALCS  2015  Spring   
4  http://matchhistory.na.leagueoflegends.com/en/...  NALCS  2015  Spring   

                                Blue                                 \
  GameType gamelength Winner TeamTag      Top  TopChamp      Jungle   
0   Season         40   Blue     TSM    Dyrus    Irelia    Santorin   
1   Season         38    Red     CST     Cris      Gnar     Impaler   
2   Season         40   Blue     WFX  Flaresz  Renekton  ShorterACE   
3   Season         41    Red     TIP     Rhux    Irelia        Rush   
4   Season         35   Blue     CLG    Benny      Gnar     Xmithie   

                                                                        \
  JungleChamp       Middle MiddleChamp         ADC  ADCChamp   Support   
0      RekSai     Bjergsen        Ahri  WildTurtle      Jinx   Lustboy   
1      Rengar        Jesiz        Ahri        Mash   Caitlyn     Sheep   
2      Rengar     Pobelter        Fizz       Altec     Sivir     Gleeb   
3    JarvanIV  XiaoWeiXiao     Leblanc      Apollo     Sivir    Adrian   
4    JarvanIV         Link   Lissandra  Doublelift  Tristana  aphromoo   

                   Red                                                  \
  SupportChamp TeamTag          Top TopChamp        Jungle JungleChamp   
0        Janna      C9        Balls     Gnar        Meteos       Elise   
1        Leona     DIG        Gamsu   Irelia       Crumbzz    JarvanIV   
2        Annie      GV     Hauntzer     Sion  Saintvicious      LeeSin   
3       Thresh      TL         Quas     Gnar    IWDominate        Nunu   
4        Janna      T8  CaliTrlolz8     Sion     Porpoise8      RekSai   

                                                                          \
     Middle MiddleChamp           ADC ADCChamp      Support SupportChamp   
0       Hai        Fizz        Sneaky    Sivir  LemonNation       Thresh   
1  Shiphtur        Azir        CoreJJ    Corki      KiWiKiD        Annie   
2     Keane        Azir           Cop    Corki   BunnyFuFuu        Janna   
3     Fenix        Lulu         KEITH   KogMaw      Xpecial        Janna   
4  Slooshi8        Lulu  Maplestreet8    Corki        Dodo8        Annie   

       Blue                                         Red                    \
      ban_1      ban_2      ban_3 ban_4 ban_5     ban_1    ban_2    ban_3   
0    Rumble   Kassadin  Lissandra   NaN   NaN  Tristana  Leblanc  Nidalee   
1  Kassadin      Sivir  Lissandra   NaN   NaN    RekSai    Janna  Leblanc   
2  JarvanIV  Lissandra   Kassadin   NaN   NaN   Leblanc      Zed   RekSai   
3     Annie  Lissandra   Kassadin   NaN   NaN    RekSai   Rumble   LeeSin   
4    Irelia   Pantheon   Kassadin   NaN   NaN    Rumble    Sivir   Rengar   

               
  ban_4 ban_5  
0   NaN   NaN  
1   NaN   NaN  
2   NaN   NaN  
3   NaN   NaN  
4   NaN   NaN

Merging the time based values. The way that gold_df is set up is a bit different from the reset of the datasets, so we will use that to out advantage.

In [89]:
type = gold_df['Type'].unique()
address = pd.DataFrame(gold_df['Address'].unique())
address.columns = pd.MultiIndex.from_product([address.columns, ['Info']])

In [90]:
for x in type:
    work = gold_df[gold_df['Type'] == x] #Sorting by type

    work_add = work[work.columns[0:2]] # Grabbing address for merge
    work = work.loc[:,['min_5','min_10','min_15','min_20']] # Grabbing the data we will actually use

    work_add.columns = pd.MultiIndex.from_product([work_add.columns, ['Info']])
    work.columns = pd.MultiIndex.from_product([work.columns, [x]])

    work = pd.concat([work_add,work],axis=1)
    work = work.swaplevel(0,1,axis=1)
    work = work.drop('Type',axis = 1, level = 1)
    work.head()
    final_df = pd.merge(final_df,work)

Filtering out the number of kills/structures destroyed/ monsters killed under 20min and adding them in to the dataset.

In [145]:
#@jit(target_backend='cuda') #for gpu calc
def TimeDataMerge(df, colnames, lvlnames):
    # df is the dataframe to add in
    # final is the final dataframe to use
    # colnames is the names of the columns for what is being added in
    # lvlnames is the same as columns but for the levels
    
    address = pd.DataFrame(df['Address'].unique())
    address.columns = [['Info'],['Address']]
    fill = pd.DataFrame(np.zeros([address.shape[0],1]))
    type = df['Team'].unique()
    final = address.copy()
    
    for x in np.arange(1,2):#len(type)):
        work = df[df['Team'] == type[x]]
        for i in np.arange(0,len(work['Address'].unique())):
            work2 = work[work['Address'] == work['Address'].unique()[i]]
            work2 = work2[work2['Time'] <= 20]
            if work2.empty == False:
                fill.loc[i] = work2.shape[0]
            else:
                fill.loc[i] = 0

        df2 = pd.concat([pd.DataFrame(work['Address'].unique(),columns=[['Info'],['Address']]),fill],
                    axis=1,ignore_index=True)
        df2.columns = [['Info',lvlnames[x]],['Address',colnames[x]]]
        final = pd.merge(final,df2)
        
    print(x) # This takes a while this is to mark down which iteration is done
    return final

In [92]:
col = ['Towers','Towers','Inhib','Inhib']
lvl = ['Blue','Red','Blue','Red']

final = TimeDataMerge(structures_df, col, lvl)
final_df = pd.merge(final_df,final)

3


In [99]:
col = ['Kills','Kills']
lvl = ['Blue','Red']
df = kills_df.drop(['x_pos','y_pos'], axis = 1)

final = TimeDataMerge(df, col, lvl)
final_df = pd.merge(final_df,final)

1


## 3) Feature Engineering

In [161]:
address = pd.DataFrame(kills_df['Address'].unique(),columns = ['Address'])
type = kills_df['Team'].unique()

In [204]:
#@jit(target_backend='cuda') #for gpu calc
def first(df):
    #df is the dataset
    first = pd.Series()
    b = pd.Series(['Blue'])
    r = pd.Series(['Red'])

    for i in np.arange(0,len(df['Address'].unique())):
        blue = df[df['Team'] == type[0]]
        red = df[df['Team'] == type[1]]

        bluefirst = blue[blue['Address'] == address['Address'].unique()[i]]
        bluefirst.reset_index(inplace = True)
        bluefirst = bluefirst['Time'][0]

        redfirst = red[red['Address'] == address['Address'].unique()[i]]
        redfirst.reset_index(inplace = True)
        redfirst = redfirst['Time'][0]

        if bluefirst < redfirst:
            first = pd.concat([first,b],axis = 0,ignore_index=True)
        else:
            first = pd.concat([first,r],axis = 0,ignore_index=True)
    
    return first

In [205]:
df = first(kills_df)

C:\Users\drago\AppData\Local\Temp/ipykernel_14552/2992577287.py:4: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  first = pd.Series()


In [153]:
df = pd.concat([address,df],axis=1)
df.columns = [['Info','Info'],['Address','FirstBlood']]
final_df = pd.merge(final_df,df)

In [154]:
final_df.head()

Info                       \
                                             Address League  Year  Season   
0  http://matchhistory.na.leagueoflegends.com/en/...  NALCS  2015  Spring   
1  http://matchhistory.na.leagueoflegends.com/en/...  NALCS  2015  Spring   
2  http://matchhistory.na.leagueoflegends.com/en/...  NALCS  2015  Spring   
3  http://matchhistory.na.leagueoflegends.com/en/...  NALCS  2015  Spring   
4  http://matchhistory.na.leagueoflegends.com/en/...  NALCS  2015  Spring   

                                Blue                                 \
  GameType gamelength Winner TeamTag      Top  TopChamp      Jungle   
0   Season         40   Blue     TSM    Dyrus    Irelia    Santorin   
1   Season         38    Red     CST     Cris      Gnar     Impaler   
2   Season         40   Blue     WFX  Flaresz  Renekton  ShorterACE   
3   Season         41    Red     TIP     Rhux    Irelia        Rush   
4   Season         35   Blue     CLG    Benny      Gnar     Xmithie   

                                                                        \
  JungleChamp       Middle MiddleChamp         ADC  ADCChamp   Support   
0      RekSai     Bjergsen        Ahri  WildTurtle      Jinx   Lustboy   
1      Rengar        Jesiz        Ahri        Mash   Caitlyn     Sheep   
2      Rengar     Pobelter        Fizz       Altec     Sivir     Gleeb   
3    JarvanIV  XiaoWeiXiao     Leblanc      Apollo     Sivir    Adrian   
4    JarvanIV         Link   Lissandra  Doublelift  Tristana  aphromoo   

                   Red                                                  \
  SupportChamp TeamTag          Top TopChamp        Jungle JungleChamp   
0        Janna      C9        Balls     Gnar        Meteos       Elise   
1        Leona     DIG        Gamsu   Irelia       Crumbzz    JarvanIV   
2        Annie      GV     Hauntzer     Sion  Saintvicious      LeeSin   
3       Thresh      TL         Quas     Gnar    IWDominate        Nunu   
4        Janna      T8  CaliTrlolz8     Sion     Porpoise8      RekSai   

                                                                          \
     Middle MiddleChamp           ADC ADCChamp      Support SupportChamp   
0       Hai        Fizz        Sneaky    Sivir  LemonNation       Thresh   
1  Shiphtur        Azir        CoreJJ    Corki      KiWiKiD        Annie   
2     Keane        Azir           Cop    Corki   BunnyFuFuu        Janna   
3     Fenix        Lulu         KEITH   KogMaw      Xpecial        Janna   
4  Slooshi8        Lulu  Maplestreet8    Corki        Dodo8        Annie   

       Blue                                         Red                    \
      ban_1      ban_2      ban_3 ban_4 ban_5     ban_1    ban_2    ban_3   
0    Rumble   Kassadin  Lissandra   NaN   NaN  Tristana  Leblanc  Nidalee   
1  Kassadin      Sivir  Lissandra   NaN   NaN    RekSai    Janna  Leblanc   
2  JarvanIV  Lissandra   Kassadin   NaN   NaN   Leblanc      Zed   RekSai   
3     Annie  Lissandra   Kassadin   NaN   NaN    RekSai   Rumble   LeeSin   
4    Irelia   Pantheon   Kassadin   NaN   NaN    Rumble    Sivir   Rengar   

              golddiff                       goldblue                         \
  ban_4 ban_5    min_5 min_10 min_15  min_20    min_5 min_10 min_15   min_20   
0   NaN   NaN     -268   -625   -790  1422.0     5068  11361  18324  29519.0   
1   NaN   NaN      147   -242   1394  1105.0     5511  11419  19698  27282.0   
2   NaN   NaN       34    913   2922  5293.0     4933  12374  21237  31317.0   
3   NaN   NaN      228     16   -335   768.0     5398  12033  19332  25772.0   
4   NaN   NaN      113    205   -574   478.0     5404  11943  19426  27472.0   

  goldred                        goldblueTop                        \
    min_5 min_10 min_15   min_20       min_5 min_10 min_15  min_20   
0    5336  11986  19114  28097.0         893   2051   3630  6008.0   
1    5364  11661  18304  26177.0        1127   2406   4594  6217.0   
2    4899  11461  18315  26024.0         828   2419   4187

In [155]:
structures_df['Team'].unique()

array(['bTowers', 'rTowers', 'bInhibs', 'rInhibs'], dtype=object)

In [156]:
address = pd.DataFrame(structures_df['Address'].unique(),columns = ['Address'])
type = structures_df['Team'].unique()
df = first(structures_df)

C:\Users\drago\AppData\Local\Temp/ipykernel_14552/3019584778.py:4: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  first = pd.Series()


In [157]:
df = pd.concat([address,df],axis=1)
df.columns = [['Info','Info'],['Address','FirstTower']]
final_df = pd.merge(final_df,df)

In [158]:
final_df.shape

(7582, 99)

## 4) Write cleaned data to csv

In [159]:
final_df.to_csv('../clean_data/data.csv')